In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [2]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [3]:
def hinge_loss(y, X, w):
    return np.clip(1 - y * (X @ w), 0, np.inf)

In [4]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    v = hinge_loss(y, X, w)
    # ***************************************************
    return np.sum(v) + lambda_ / 2 * np.sum(w ** 2)

In [5]:
def accuracy(y1, y2):
    return np.mean(y1 == y2)

def prediction(X, w):
    return (X @ w > 0) * 2 - 1

def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    predicted_y = prediction(X, w)
    # ***************************************************
    return accuracy(predicted_y, y)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [6]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    def is_support(y_n, x_n, w):
        """a datapoint is support if max{} is not 0. """
        return y_n * x_n @ w < 1
    
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    grad = - y_n * x_n.T if is_support(y_n, x_n, w) else np.zeros_like(x_n.T)
    grad = num_examples * np.squeeze(grad) + lambda_ * w
    return grad

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [7]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=260609021.0721839
iteration=10000, cost=741.9220036339556
iteration=20000, cost=182.98957002706783
iteration=30000, cost=261.5382194313897
iteration=40000, cost=173.79960528764943
iteration=50000, cost=155.0807709744077
iteration=60000, cost=146.81062718482028
iteration=70000, cost=131.5570399215903
iteration=80000, cost=137.13318286888105
iteration=90000, cost=131.75883676477275
iteration=100000, cost=152.98579036374397
iteration=110000, cost=150.02833618954836
iteration=120000, cost=130.60077856109424
iteration=130000, cost=133.2312948766016
iteration=140000, cost=125.79379878268254
iteration=150000, cost=131.9837005586306
iteration=160000, cost=124.65051456491638
iteration=170000, cost=124.51683681772016
iteration=180000, cost=125.8886412714957
iteration=190000, cost=124.79263732869389
training accuracy = 0.9121265377855887


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [8]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    g = (1 - y_n * x_n.dot(w))

    if g != 0:
        alpha[n] = min(
            max(old_alpha_n + lambda_ * g / (x_n.T.dot(x_n)), 0.0),
            1.0)

        # compute the corresponding update on the primal vector w
        w += 1.0 / lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

In [9]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    # ***************************************************
    return np.sum(alpha)  - lambda_ / 2.0 * np.sum(w ** 2) # w = 1/lambda * X * Y * alpha
    # ***************************************************

In [10]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:588.15902, dual:0.00366, gap:588.15536
iteration=10000, primal:147.41417, dual:31.71245, gap:115.70172
iteration=20000, primal:138.71796, dual:55.46074, gap:83.25721
iteration=30000, primal:329.72177, dual:74.44393, gap:255.27784
iteration=40000, primal:129.22738, dual:88.86695, gap:40.36043
iteration=50000, primal:133.30813, dual:98.14487, gap:35.16326
iteration=60000, primal:139.46137, dual:104.28258, gap:35.17879
iteration=70000, primal:125.01563, dual:108.39281, gap:16.62282
iteration=80000, primal:126.58272, dual:111.52497, gap:15.05774
iteration=90000, primal:126.46074, dual:113.73822, gap:12.72252
iteration=100000, primal:123.24434, dual:115.46398, gap:7.78037
iteration=110000, primal:131.55592, dual:116.78756, gap:14.76836
iteration=120000, primal:129.95747, dual:117.79495, gap:12.16252
iteration=130000, primal:122.85394, dual:118.69229, gap:4.16164
iteration=140000, primal:125.55600, dual:119.43037, gap:6.12562
iteration=150000, primal:124.14410, dual:120.0